In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [2]:
my_collection = "Training on Python, Spark, Scala, ML"\
  .split(" ")
words = spark.sparkContext.parallelize(my_collection, 2)

In [8]:
supplementalData = {"Spark":1000, "Training":200,
                    "Big":-300, "Simple":100}

In [9]:
suppBroadcast = spark.sparkContext.broadcast(supplementalData)

In [10]:
suppBroadcast.value

{'Spark': 1000, 'Training': 200, 'Big': -300, 'Simple': 100}

In [11]:
words.map(lambda word: (word, suppBroadcast.value.get(word, 0)))\
  .sortBy(lambda wordPair: wordPair[1])\
  .collect()

[('on', 0),
 ('Python,', 0),
 ('Spark,', 0),
 ('Scala,', 0),
 ('ML', 0),
 ('Training', 200)]

In [21]:
!dir "..\Day3-Day4\data\flight-data\parquet\2010-summary.parquet"

 Volume in drive D is Windows
 Volume Serial Number is 949A-70EE

 Directory of D:\22-Trngs\2-Confirmed\5-PySpark-56-hours-Geet\GH\Labs\Day3-Day4\data\flight-data\parquet\2010-summary.parquet

12-05-2021  15:40    <DIR>          .
12-05-2021  15:40    <DIR>          ..
12-05-2021  15:40             3,921 part-r-00000-1a9822ba-b8fb-4d8e-844a-ea30d0801b9e.gz.parquet
12-05-2021  15:40                 0 _SUCCESS
               2 File(s)          3,921 bytes
               2 Dir(s)  1,790,909,059,072 bytes free


In [23]:
flights = spark.read\
  .parquet("../Day3-Day4/data/flight-data/parquet/2010-summary.parquet")

In [24]:
accChina = spark.sparkContext.accumulator(0)

In [25]:
def accChinaFunc(flight_row):
  destination = flight_row["DEST_COUNTRY_NAME"]
  origin = flight_row["ORIGIN_COUNTRY_NAME"]
  if destination == "China":
    accChina.add(flight_row["count"])
  if origin == "China":
    accChina.add(flight_row["count"])

In [26]:
flights.foreach(lambda flight_row: accChinaFunc(flight_row))

In [27]:
accChina.value # 953

953